# Instructions

Run the code and say a word from the list. The model will repeat the word back to you. If you say stop, the code will stop running

In [57]:
import tensorflow as tf
import pyaudio
import numpy as np

# Parameters for audio capture
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
CHUNK_SIZE = 16000

label_names = ['down', 'go', 'left', 'no', 'right', 'stop', 'up', 'yes']

# Initialize PyAudio
p = pyaudio.PyAudio()

# Open a stream for audio capture
stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK_SIZE)

print("Capturing audio...")



# Load the SavedModel
model = tf.saved_model.load("saved")

try:
    while True:
        # Read audio data from the stream
        data = np.fromstring(stream.read(CHUNK_SIZE), dtype=np.int16)
        
        # Convert the data to numpy array for further processing
        audio_data = tf.convert_to_tensor(data, dtype=tf.float32)
        audio_data = np.expand_dims(audio_data, axis=0)
        
        # Feed audio through model
        y_pred = model(audio_data)
        y_pred = y_pred['class_ids'].numpy()[0]
        command = label_names[y_pred]

        rms_amplitude = np.sqrt(np.mean(np.square(audio_data)))

        if command == 'stop' and rms_amplitude > 500:
            print("Stopping audio capture...")
            # Close the stream and terminate PyAudio
            stream.stop_stream()
            stream.close()
            p.terminate()
        else:
            if rms_amplitude > 500:
                print("RMS Amplitude:", rms_amplitude)
                print(command)
        
        rms_amplitude = 0

        
except KeyboardInterrupt:
    print("Stopping audio capture...")
    # Close the stream and terminate PyAudio
    stream.stop_stream()
    stream.close()
    p.terminate()


Capturing audio...


C:\Users\lucas\AppData\Local\Temp\ipykernel_29088\2620160804.py:36: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  data = np.fromstring(stream.read(CHUNK_SIZE), dtype=np.int16)


RMS Amplitude: 1916.4381
right
RMS Amplitude: 1531.6527
yes
RMS Amplitude: 2111.777
yes
RMS Amplitude: 5320.574
down
RMS Amplitude: 10244.967
go
RMS Amplitude: 3456.8477
up
RMS Amplitude: 2668.3184
down
RMS Amplitude: 2768.1492
left
RMS Amplitude: 2181.8079
right
RMS Amplitude: 2569.1086
up
RMS Amplitude: 727.5566
right
RMS Amplitude: 2265.7942
up
RMS Amplitude: 726.19226
yes
RMS Amplitude: 1776.0574
up
Stopping audio capture...


OSError: [Errno -9988] Stream closed